In [21]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import trange

DATA_DIR = Path(".") / "data"

print("Libraries loaded. Ready to solve TSP using Greedy + Evolution Strategy.")


Libraries loaded. Ready to solve TSP using Greedy + Evolution Strategy.


In [27]:

def tour_cost(matrix: np.ndarray, tour: np.ndarray) -> float:
    """Compute cost of a tour (vectorized)."""
    return np.sum(matrix[tour, np.roll(tour, -1)])


def two_opt(tour: np.ndarray, matrix: np.ndarray) -> np.ndarray:
    """Simple 2-opt local search improvement."""
    improved = True
    best_cost = tour_cost(matrix, tour)
    while improved:
        improved = False
        for i in range(1, len(tour) - 2):
            for j in range(i + 1, len(tour)):
                if j - i == 1:
                    continue
                new_tour = np.concatenate((tour[:i], tour[i:j][::-1], tour[j:]))
                new_cost = tour_cost(matrix, new_tour)
                if new_cost < best_cost:
                    tour, best_cost = new_tour, new_cost
                    improved = True
        if improved is False:
            break
    return tour


In [28]:

def greedy_solver(matrix: np.ndarray, seed: int = 42, restarts: int = 5, use_2opt: bool = True) -> tuple:
    """Construct tour using Greedy heuristic (nearest neighbor) with optional restarts."""
    rng = np.random.default_rng(seed)
    n = matrix.shape[0]
    best_tour = None
    best_cost = float("inf")
    history = []

    for _ in range(restarts):
        start = rng.integers(0, n)
        unvisited = list(range(n))
        tour = [start]
        unvisited.remove(start)

        while unvisited:
            current = tour[-1]
            next_city = min(unvisited, key=lambda city: matrix[current, city])
            tour.append(next_city)
            unvisited.remove(next_city)

        tour = np.array(tour)
        if use_2opt:
            tour = two_opt(tour, matrix)

        cost = tour_cost(matrix, tour)
        history.append(cost)
        if cost < best_cost:
            best_cost, best_tour = cost, tour

    return best_tour, history


In [45]:

from dataclasses import dataclass

@dataclass
class ESParams:
    mu: int = 30
    lam: int = 100
    generations: int = 500
    mutation_rate: float = 0.3
    greedy_fraction: float = 0.7
    seed: int = 42


def evolutionary_strategy(matrix: np.ndarray, params: ESParams, warm_start: np.ndarray | None = None):
    rng = np.random.default_rng(params.seed)
    n = matrix.shape[0]
    population = [rng.permutation(n) for _ in range(params.mu)]

    if warm_start is not None:
        population[0] = warm_start.copy()

    cost_cache = {}
    def get_cost(tour):
        key = tuple(tour)
        if key not in cost_cache:
            cost_cache[key] = tour_cost(matrix, tour)
        return cost_cache[key]

    costs = np.array([get_cost(ind) for ind in population])
    history = [float(costs.min())]

    best_cost = history[-1]
    no_improve = 0

    for gen in trange(params.generations, desc="Evolution Strategy"):
        offspring = []
        for _ in range(params.lam):
            parent_idx = rng.integers(0, params.mu)
            child = population[parent_idx].copy()
            if rng.random() < params.mutation_rate:
                i, j = rng.choice(n, size=2, replace=False)
                child[i:j] = child[i:j][::-1]
            offspring.append(child)

        all_candidates = population + offspring
        all_costs = np.array([get_cost(ind) for ind in all_candidates])

        parents_idx = np.argpartition(all_costs, params.mu)[:params.mu]
        population = [all_candidates[i] for i in parents_idx]
        costs = all_costs[parents_idx]

        history.append(float(costs.min()))

        # Early stopping
        if history[-1] < best_cost - 1e-6:
            best_cost = history[-1]


    best_idx = int(np.argmin(costs))
    return population[best_idx].copy(), history


In [41]:

def run_pipeline(matrix: np.ndarray, seed: int = 42) -> dict:
   # greedy_tour, greedy_hist = greedy_solver(matrix, seed=seed)
    es_tour, es_hist = evolutionary_strategy(matrix, ESParams(seed=seed))

    leaderboard = [
        #("Greedy2Opt", greedy_tour, greedy_hist[-1]),
        ("ES_mu_plus_lambda", es_tour, es_hist[-1]),
    ]
    winner = min(leaderboard, key=lambda item: item[2])

    return {
        #"greedy": (greedy_tour, greedy_hist),
        "es": (es_tour, es_hist),
        "winner": winner,
    }


In [46]:

def evaluate_instances(pattern: str = "problem_*.npy", seed: int = 42, fast: bool = True, limit: int | None = None) -> pd.DataFrame:
    paths = sorted(DATA_DIR.glob(pattern))
    if limit:
        paths = paths[:limit]

    rows = []
    for idx, path in enumerate(paths, 1):
        print(f"[{idx}/{len(paths)}] {path.name}...", end=" ", flush=True)
        matrix = np.load(path)
        n = matrix.shape[0]

        #greedy_tour, greedy_hist = greedy_solver(matrix, seed=seed)
        es_tour, es_hist = evolutionary_strategy(matrix, ESParams(seed=seed))

        leaderboard = [
            
            ("ES_mu_plus_lambda", es_hist[-1]),
        ]
        winner_name, winner_cost = min(leaderboard, key=lambda item: item[1])

        row = {
            "instance": path.name,
            "size": n,
            "es_cost": es_hist[-1],
            "winner": winner_name,
            "winner_cost": winner_cost,
        }
        rows.append(row)
        print(f"✓ {winner_name}: {winner_cost:.2f}")

    return pd.DataFrame(rows)


print("Running (ES only)...")
leaderboard_df = evaluate_instances(fast=True, limit=22)
print("\nResults:")
print(leaderboard_df[["instance", "size", "winner", "winner_cost"]].to_string(index=False))

leaderboard_df.to_csv("results/hybrid_leaderboard_optimized.csv", index=False)
print("\nSaved to results/hybrid_leaderboard_optimized.csv")


Running (ES only)...
[1/21] problem_g_10.npy... 

Evolution Strategy: 100%|██████████| 500/500 [00:00<00:00, 507.87it/s]

✓ ES_mu_plus_lambda: 1497.66
[2/21] problem_g_100.npy... 


Evolution Strategy: 100%|██████████| 500/500 [00:01<00:00, 303.68it/s]

✓ ES_mu_plus_lambda: 6333.92
[3/21] problem_g_1000.npy... 

...

✓ ES_mu_plus_lambda: -3600.35
[20/21] problem_r2_50.npy... 


Evolution Strategy: 100%|██████████| 500/500 [00:01<00:00, 301.64it/s]

✓ ES_mu_plus_lambda: -1252.98
[21/21] problem_r2_500.npy... 


Evolution Strategy: 100%|██████████| 500/500 [00:05<00:00, 92.64it/s]

✓ ES_mu_plus_lambda: -5257.18

Results:
           instance  size            winner   winner_cost
   problem_g_10.npy    10 ES_mu_plus_lambda   1497.663648
  problem_g_100.npy   100 ES_mu_plus_lambda   6333.917072
 problem_g_1000.npy  1000 ES_mu_plus_lambda 187316.568075
   problem_g_20.npy    20 ES_mu_plus_lambda   1787.878418
  problem_g_200.npy   200 ES_mu_plus_lambda  17562.388504
   problem_g_50.npy    50 ES_mu_plus_lambda   2862.528568
  problem_g_500.npy   500 ES_mu_plus_lambda  74044.027009
  problem_r1_10.npy    10 ES_mu_plus_lambda    241.116629
 problem_r1_100.npy   100 ES_mu_plus_lambda   2301.097899
problem_r1_1000.npy  1000 ES_mu_plus_lambda  45106.661573
  problem_r1_20.npy    20 ES_mu_plus_lambda    393.897945
 problem_r1_200.npy   200 ES_mu_plus_lambda   6546.461231
  problem_r1_50.npy    50 ES_mu_plus_lambda   1061.579662
 problem_r1_500.npy   500 ES_mu_plus_lambda  19655.244851
  problem_r2_10.npy    10 ES_mu_plus_lambda   -313.772944
 problem_r2_100.npy   100 ES_mu_

In [ ]:
ourCities = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('data/test_problem.npy')